In [10]:
%matplotlib inline
from pylab import *
from ipywidgets import widgets
# [widget for widget in dir(widgets) if widget.endswith('Widget')]

### Importing

In [11]:
import os, sys

In [12]:
sys.path.append('/home/mag/Documents/repos/solab/posada/')

In [13]:
import CommonSAR
reload(CommonSAR)
import pySAR
reload(pySAR)
import pySAR

### Default values

In [14]:
iPath = '/nfs1/store/satellite/asar/2012/036'
fileName = 'ASA_WSM_1PNPDK20120205_093002_000000923111_00223_51961_2779.N1'
pxlRes   = 800
image_size_skip = 140*1e6
area_bbox = 35
units     = 'dB'
oPath     = '/media/SOLabNFS/hyrax-cluster/data/public/allData/tzh/'
# When False append to NC wind and roughness and skip appending sigma0?
overwrite_nc = True
asar_conf = '/home/mag/Documents/repos/solab/PySOL/notebooks/pySAR/ASAR.json'
proj      = 'EPSG:3413'
numProcs  = 8

### Computaions

In [15]:
asar = pySAR.pySAR(iPath, fileName, pxlRes, image_size_skip, area_bbox, units, oPath, overwrite_nc, asar_conf, proj, numProcs)

In [16]:
asar.computeSARwind()

Calculating CMOD...
Sigma to Wind LUT...
CMOD elapsed time: 0.386897


In [17]:
# asar.resampleSAR(var_name=['sigma0'])
asar.resampleSAR(var_name=['sigma0', 'wind_speed'])

In [18]:
# asar.generateTiles(var_name=['sigma0'])
asar.generateTiles(var_name=['sigma0', 'wind_speed'])

In [ ]:
print asar.startTime

# print 'http://earth.nullschool.net/#'+startTime.strftime("%Y/%m/%d/")+'%.2d' %(ncepGFSmodelWind['baseHour'])+'00Z/wind/surface/level/orthographic=-327.02,60.84,3000'
print 'http://earth.nullschool.net/#'+asar.startTime.strftime("%Y/%m/%d/")+'%.2d' %(asar.ncepGFSmodelWind['baseHour'])+'00Z/wind/surface/level/orthographic='+str(asar.lonlim[0])+','+str(asar.latlim[0])+',3000'

### Interactive plotting

In [ ]:
# from IPython.html import widgets
from ipywidgets import widgets, interact
# [widget for widget in dir(widgets) if widget.endswith('Widget')]

In [ ]:
# %matplotlib inline
def ntrctv_imshow(p = 'hh', vmi=-1., vma=1., cmap='bone', crop='[:,:]'):
    # check if data type is dictionary and there is no wind
    if type(data) == dict and all(k!='wind_speed' for k in data.iterkeys()):
        plt.figure(figsize=(16,16*double(data[p].shape[0])/double(data[p].shape[1])))
        plt.imshow(eval("data[p]"+str(crop)), vmin=vmi, vmax=vma)
    elif type(data) == dict and any(k=='wind_speed' for k in data.iterkeys()):
        plt.figure(figsize=(16,16*double(data[p].shape[0])/double(data[p].shape[1])))
        X,Y = meshgrid( arange(0,eval("data[p]"+str(crop)).shape[1]),arange(0,eval("data[p]"+str(crop)).shape[0]) )
        U = (data['u'])
        V = (data['v'])
        U = eval("U"+str(crop))
        V = eval("V"+str(crop))
        scl = 100
        plt.quiver(X[::scl,::scl], Y[::scl,::scl], U[::scl,::scl], V[::scl,::scl])
        plt.imshow(eval("data[p]"+str(crop)), vmin=vmi, vmax=vma)
        plt.axis('tight')
    elif type(data) != dict:
        plt.figure(figsize=(16,16*double(data.shape[0])/double(data.shape[1])))
        plt.imshow(eval("data"+str(crop)), vmin=vmi, vmax=vma)
    plt.colorbar()
    plt.set_cmap(cmap)

In [ ]:
from scipy.stats import scoreatpercentile

data=10*log10(asar.sigma0)

vmin = None
vmax = None
# vmin_r = None
# vmax_r = None

if vmin is None:
    vmin = scoreatpercentile(data[25:-25, 25:-25], 1)
if vmax is None:
    vmax = scoreatpercentile(data[25:-25, 25:-25], 99.9)

# if vmin_r is None:
#     vmin_r = scoreatpercentile(roughness[p][25:-25, 25:-25], 1)
# if vmax_r is None:
#     vmax_r = scoreatpercentile(roughness[p][25:-25, 25:-25], 99.9)

In [ ]:

ntrctv = interact(ntrctv_imshow, \
                 vmi=widgets.FloatSlider(min=-35, max=5, value=vmin, step=0.5), \
                 vma=widgets.FloatSlider(min=-35, max=5, value=vmax, step=0.5), \
                cmap = ['bone', 'gray', 'RdBu_r'], crop = '[:,:]');

In [ ]:
# data=asar.ncepGFSmodelWind
data = asar.wind_speed_asar

# ntrctv = interact(ntrctv_imshow, p = widgets.RadioButtons(description='wind speed', options=data.keys()), \
ntrctv = interact(ntrctv_imshow, \
                 vmi=widgets.FloatSlider(min=0, max=30, value=0, step=1), \
                 vma=widgets.FloatSlider(min=0, max=30, value=15., step=1), \
                cmap = ['jet'], crop = '[:,:]');
# crop = '[600:1400,0:900]'